In [125]:
from ib_async import *
import pandas as pd
from datetime import datetime, date, timedelta, time
import numpy as np

util.startLoop()

In [4]:
ib = IB()
ib.connect(port=4002, clientId=4)

<IB connected to 127.0.0.1:4002 clientId=4>

In [5]:
contract = Contract(symbol='META', secType='STK', exchange='SMART', currency='USD')
ib.qualifyContracts(contract)

[Contract(secType='STK', conId=107113386, symbol='META', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='META', tradingClass='NMS')]

In [166]:
class Data_Request :

    """
    Returns PD data containning equity price with specific time cut-off

    Args:
    kwargs: specific-time to cut-off

    Returns:
        pd: information regarding the contract of interst.
    """
    
    def __init__(self, list_days, contract, barSize, duration, *,specific_time= None, specific_pick_time= None):

        self.list_days = list_days
        self.contract = contract
        #self.init_date = init_date
        self.barsize = barsize
        self.duration = duration
        self.specific_time = specific_time # cut off time
        self.specific_pick_time= specific_pick_time

    def D_request(self):

        # specific_time = None
        
        # if "specific_time" in self.kwargs.keys():
        #     specific_time = self.kwargs["specific_time"]
            
        # for key, value in kwargs.items():
            
        # print(f"{key}: {value}")

        his_df =pd.DataFrame()
        #specific_time = time(14, 30, 0)  # cut-off time.. xargs.. 
        
        for i in range(len(self.list_days)):
            init_date=self.list_days[i]
            bars = ib.reqHistoricalData(self.contract, init_date, barSizeSetting=self.barsize, durationStr=self.duration, whatToShow="TRADES", useRTH=True)
            add_his_df = pd.DataFrame(bars)
            add_his_df['date'] = pd.to_datetime(add_his_df['date'])
            print(f'retriveing data for {init_date}')

            if self.specific_time is not None:
                cut_off = datetime.combine(self.list_days[i],self.specific_time)  # need logic if specific_time is defined
                pd_cut_off = pd.to_datetime(cut_off).tz_localize('US/Eastern')
            #datetime64_eastern = pd.Timestamp(date_time_str).tz_localize('US/Eastern')
                add_his_df = add_his_df[add_his_df['date'] > pd_cut_off]
            
            if self.specific_pick_time is not None:
                pick_time = datetime.combine(self.list_days[i],self.specific_pick_time)  # need logic if specific_time is defined
                pd_pick_time = pd.to_datetime(pick_time).tz_localize('US/Eastern')
            #datetime64_eastern = pd.Timestamp(date_time_str).tz_localize('US/Eastern')
                add_his_df = add_his_df[add_his_df['date'] == pd_pick_time]

            print(len(his_df))
            his_df = pd.concat([his_df, add_his_df], ignore_index=True)
            hid_df['Date_Only'] = his_df['date'].dt.date
        return his_df


In [281]:
def get_fridays_in_range(start_date, end_date):
    """
    Returns a list of dates that are Fridays within the given date range.

    Args:
        start_date (date): The start date of the range.
        end_date (date): The end date of the range.

    Returns:
        list: A list of date objects representing Fridays within the range.
    """
    fridays = []
    all_dates = []
    current_date = start_date
    all_current_date = start_date

    # Adjust to the first Friday
    days_until_friday = (4 - current_date.weekday()) % 7
    current_date += timedelta(days=days_until_friday)

    while current_date <= end_date:
        fridays.append(current_date)
        current_date += timedelta(days=7) # Move to the next Friday

    while all_current_date <= end_date:
        all_dates.append(all_current_date)
        all_current_date += timedelta(days=1)
    
    return fridays, all_dates

# Example Usage
start_date = date(2024, 1, 1)
end_date = date(2025, 3, 30)

fridays, all_dates = get_fridays_in_range(start_date, end_date)

In [282]:
len(fridays)

65

In [283]:
# removing Non-trading Holidays
fridays.remove(date(2024, 3, 29))
fridays.remove(date(2024, 11, 29))

In [284]:
len(fridays)

63

In [ ]:
### Meta stock historical data requst ##

In [ ]:
barsize ="1 min"
duration = "1 D"
DR_stock_meta = Data_Request(all_dates, contract, barsize, duration)

In [ ]:
# only when making API call to RECOLLECT
# df_all_meta_stock = DR_stock_meta.D_request()

In [ ]:
df_all_meta_stock.to_csv('meta_stock_info.csv', index=False)
df_all_meta_stock.to_pickle('meta_stock_info.pkl')

In [12]:
#!pip install yfinance

     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     -------------------- ------------------- 1.6/3.0 MB 7.7 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 8.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for peewee: filename=peewee-3.17.9-py3-none-any.whl size=139127 sha256=d3df6543407d320d9e629599efe80dc8cd8a9849dc3850eff423a7c3e4a56b8f
  Stored in directory: c:\users\jepar\appdata\local\pip\cache\wheels\43\ef\2d\2c51d496bf084945ffdf838b4cc8767b8ba1cc20eb41588831
Successfully built peewee


In [17]:
# Import yfinance package
import yfinance as yf

# Set the ticker
ticker = '^IXIC'

# Get the data
Nasdaq_data = yf.download(ticker, start_date, end_date)

# Print 5 rows
Nasdaq_data

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,^IXIC,^IXIC,^IXIC,^IXIC,^IXIC
Date,,,,,
2024-01-02,14765.940430,14887.799805,14682.379883,14873.700195,5791170000
2024-01-03,14592.209961,14694.580078,14577.440430,14641.469727,5449950000
2024-01-04,14510.299805,14632.769531,14504.780273,14532.230469,5289220000
2024-01-05,14524.070312,14625.190430,14477.570312,14500.110352,5232820000
2024-01-08,14843.769531,14846.900391,14560.200195,14564.469727,6337770000
...,...,...,...,...,...
2025-03-24,18188.589844,18210.050781,18030.679688,18046.189453,6787790000
2025-03-25,18271.859375,18281.130859,18170.419922,18207.970703,6475790000


In [ ]:
# taking 1 day - 3 day price movement

In [378]:
Nasdaq_data['Nas_1d_diff'] = Nasdaq_data['Open'].diff()

In [379]:
Nasdaq_data['Nas_2d_diff'] = Nasdaq_data['Open'].diff(periods=2)

In [380]:
Nasdaq_data['Nas_3d_diff'] = Nasdaq_data['Open'].diff(periods=3)

In [ ]:
# VIX index info

In [26]:
# Set the ticker
ticker = '^VIX'

# Get the data
Vix_data = yf.download(ticker, start_date, end_date)

# Print 5 rows
Vix_data

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,^VIX,^VIX,^VIX,^VIX,^VIX
Date,,,,,
2024-01-02,13.200000,14.230000,13.100000,13.220000,0
2024-01-03,14.040000,14.220000,13.330000,13.350000,0
2024-01-04,14.130000,14.200000,13.640000,13.930000,0
2024-01-05,13.350000,14.580000,13.290000,14.240000,0
2024-01-08,13.080000,14.180000,13.020000,14.000000,0
...,...,...,...,...,...
2025-03-24,17.480000,19.139999,17.459999,19.129999,0
2025-03-25,17.150000,17.770000,17.020000,17.580000,0


In [ ]:
# getting daily META data to get 1d- 3d price movement

In [62]:
# Set the ticker
ticker = 'META'

# Get the data
Meta_data = yf.download(ticker, start_date, end_date)

# Print 5 rows
Meta_data

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,META,META,META,META,META
Date,,,,,
2024-01-02,344.665619,351.503388,338.415079,349.672023,19042200
2024-01-03,342.854156,346.317843,341.570199,343.361774,15451100
2024-01-04,345.491699,346.516866,341.789148,342.883994,12099900
2024-01-05,350.299042,351.841759,344.635731,345.362287,13920700
2024-01-08,356.977539,357.296045,350.398531,353.036124,13890200
...,...,...,...,...,...
2025-03-24,618.849976,622.539978,612.200012,614.969971,15741300
2025-03-25,626.309998,633.880005,621.179993,626.760010,15312500


In [63]:
Meta_data['Diff_Price'] = Meta_data['Close'] - Meta_data['Open']

In [64]:
Meta_data['Vix'] = Vix_data['Close']

In [65]:
Meta_data['Nas_open']= Nasdaq_data['Open']
Meta_data['Nas_close']= Nasdaq_data['Close']
Meta_data['Nas_diff']= Nasdaq_data['Close'] - Nasdaq_data['Open']

In [66]:
Meta_

Price,Close,High,Low,Open,Volume,Diff_Price,Vix,Nas_open,Nas_close,Nas_diff
Ticker,META,META,META,META,META,,,,,
Date,,,,,,,,,,
2024-01-02,344.665619,351.503388,338.415079,349.672023,19042200,-5.006404,13.200000,14873.700195,14765.940430,-107.759766
2024-01-03,342.854156,346.317843,341.570199,343.361774,15451100,-0.507617,14.040000,14641.469727,14592.209961,-49.259766
2024-01-04,345.491699,346.516866,341.789148,342.883994,12099900,2.607705,14.130000,14532.230469,14510.299805,-21.930664
2024-01-05,350.299042,351.841759,344.635731,345.362287,13920700,4.936755,13.350000,14500.110352,14524.070312,23.959961
2024-01-08,356.977539,357.296045,350.398531,353.036124,13890200,3.941415,13.080000,14564.469727,14843.769531,279.299805
...,...,...,...,...,...,...,...,...,...,...
2025-03-24,618.849976,622.539978,612.200012,614.969971,15741300,3.880005,17.480000,18046.189453,18188.589844,142.400391
2025-03-25,626.309998,633.880005,621.179993,626.760010,15312500,-0.450012,17.150000,18207.970703,18271.859375,63.888672


In [67]:
Meta_data['Diff_Price'].corr(Meta_data['Nas_diff'])

0.6947405456378072

In [68]:
Meta_data['diff_1_Open'] = Meta_data['Open'].diff(1)
#Meta_data['diff_2_Open']=Meta_data['Open'].diff(2)
#Meta_data['diff_3_Open']=Meta_data['Open'].diff(3)

In [70]:
Meta_data['diff_2_Open']=Meta_data['Open'].diff(2)

In [72]:
Meta_data['diff_3_Open']=Meta_data['Open'].diff(3)

In [74]:
Meta_data['Nas_volume'] = Nasdaq_data['Volume']

In [76]:
Features = Meta_data.copy()

In [77]:
Features

Price,Close,High,Low,Open,Volume,Diff_Price,Vix,Nas_open,Nas_close,Nas_diff,diff_1_Open,diff_2_Open,diff_3_Open,Nas_volume
Ticker,META,META,META,META,META,,,,,,,,,
Date,,,,,,,,,,,,,,
2024-01-02,344.665619,351.503388,338.415079,349.672023,19042200,-5.006404,13.200000,14873.700195,14765.940430,-107.759766,NaN,NaN,NaN,5791170000
2024-01-03,342.854156,346.317843,341.570199,343.361774,15451100,-0.507617,14.040000,14641.469727,14592.209961,-49.259766,-6.310249,NaN,NaN,5449950000
2024-01-04,345.491699,346.516866,341.789148,342.883994,12099900,2.607705,14.130000,14532.230469,14510.299805,-21.930664,-0.477780,-6.788029,NaN,5289220000
2024-01-05,350.299042,351.841759,344.635731,345.362287,13920700,4.936755,13.350000,14500.110352,14524.070312,23.959961,2.478293,2.000513,-4.309736,5232820000
2024-01-08,356.977539,357.296045,350.398531,353.036124,13890200,3.941415,13.080000,14564.469727,14843.769531,279.299805,7.673837,10.152130,9.674350,6337770000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-24,618.849976,622.539978,612.200012,614.969971,15741300,3.880005,17.480000,18046.189453,18188.589844,142.400391,31.549988,32.209961,30.689941,6787790000
2025-03-25,626.309998,633.880005,621.179993,626.760010,15312500,-0.450012,17.150000,18207.970703,18271.859375,63.888672,11.790039,43.340027,44.000000,6475790000


In [ ]:
# reading input data

In [79]:
df_all_meta_stock = pd.read_pickle('meta_stock_info.pkl')

In [80]:
df_all_meta_stock

,date,open,high,low,close,volume,average,barCount
0,2023-12-29 09:30:00-05:00,358.99,360.00,358.99,359.88,155021.0,359.308,510
1,2023-12-29 09:31:00-05:00,359.82,359.82,359.27,359.71,38832.0,359.594,254
2,2023-12-29 09:32:00-05:00,359.69,359.72,358.33,358.48,67898.0,359.113,374
3,2023-12-29 09:33:00-05:00,358.62,358.72,357.66,357.94,30127.0,358.207,180
4,2023-12-29 09:34:00-05:00,357.83,357.89,357.25,357.28,24586.0,357.559,153
...,...,...,...,...,...,...,...,...
176185,2025-03-28 15:55:00-04:00,576.47,576.80,575.30,576.63,62266.0,576.233,414
176186,2025-03-28 15:56:00-04:00,576.63,576.75,576.20,576.55,39346.0,576.498,288
176187,2025-03-28 15:57:00-04:00,576.66,577.91,576.60,577.39,85937.0,577.415,607
176188,2025-03-28 15:58:00-04:00,577.39,577.39,577.00,577.14,83324.0,577.158,654


In [81]:
df_all_meta_stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176190 entries, 0 to 176189
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype                     
---  ------    --------------   -----                     
 0   date      176190 non-null  datetime64[ns, US/Eastern]
 1   open      176190 non-null  float64                   
 2   high      176190 non-null  float64                   
 3   low       176190 non-null  float64                   
 4   close     176190 non-null  float64                   
 5   volume    176190 non-null  float64                   
 6   average   176190 non-null  float64                   
 7   barCount  176190 non-null  int64                     
dtypes: datetime64[ns, US/Eastern](1), float64(6), int64(1)
memory usage: 10.8 MB


In [358]:
# picking out Friday and before specific-time data

#specific_time = time(14, 30, 0)
def data_filter(df_initial, list_days, specific_time_before = None, specific_time_after= None, on_time= None):
    his_df = pd.DataFrame()
    for i in range(len(list_days)):
        init_date=list_days[i]
        if specific_time_before is not None:
            add_his_df = df_initial[(df_initial['date'].dt.date == init_date) & (df_initial['date'].dt.time < specific_time_before)]
           
            
        if specific_time_after is not None:
            add_his_df = df_initial[(df_initial['date'].dt.date == init_date) & (df_initial['date'].dt.time >= specific_time_after)]

        if on_time is not None:
            add_his_df = df_initial[(df_initial['date'].dt.date == init_date) & (df_initial['date'].dt.time == on_time)]

        if (specific_time_before is None and specific_time_after is None) and on_time is None:
            add_his_df = df_initial[(df_initial['date'].dt.date == init_date)]
        
        print(len(his_df))
        print(f'retrieving {init_date}')
        per_proc = np.round(i/len(list_days)*100, decimals=1)
        print(f'{per_proc} percent is done')
        his_df = pd.concat([his_df, add_his_df], ignore_index=True)
        his_df.drop_duplicates(subset=None, keep='first', inplace=True)
        his_df['Date_Only'] = his_df['date'].dt.date
    return his_df

In [285]:
df_friday_openning = data_filter(df_all_meta_stock, fridays, on_time = time(9, 30, 0) )

0
retrieving 2024-01-05
0.0 percent is done
1
retrieving 2024-01-12
1.6 percent is done
2
retrieving 2024-01-19
3.2 percent is done
3
retrieving 2024-01-26
4.8 percent is done
4
retrieving 2024-02-02
6.3 percent is done
5
retrieving 2024-02-09
7.9 percent is done
6
retrieving 2024-02-16
9.5 percent is done
7
retrieving 2024-02-23
11.1 percent is done
8
retrieving 2024-03-01
12.7 percent is done
9
retrieving 2024-03-08
14.3 percent is done
10
retrieving 2024-03-15
15.9 percent is done
11
retrieving 2024-03-22
17.5 percent is done
12
retrieving 2024-04-05
19.0 percent is done
13
retrieving 2024-04-12
20.6 percent is done
14
retrieving 2024-04-19
22.2 percent is done
15
retrieving 2024-04-26
23.8 percent is done
16
retrieving 2024-05-03
25.4 percent is done
17
retrieving 2024-05-10
27.0 percent is done
18
retrieving 2024-05-17
28.6 percent is done
19
retrieving 2024-05-24
30.2 percent is done
20
retrieving 2024-05-31
31.7 percent is done
21
retrieving 2024-06-07
33.3 percent is done
22
re

In [287]:
df_friday_cut_off = data_filter(df_all_meta_stock, fridays, on_time = time(14, 28, 0) )

0
retrieving 2024-01-05
0.0 percent is done
1
retrieving 2024-01-12
1.6 percent is done
2
retrieving 2024-01-19
3.2 percent is done
3
retrieving 2024-01-26
4.8 percent is done
4
retrieving 2024-02-02
6.3 percent is done
5
retrieving 2024-02-09
7.9 percent is done
6
retrieving 2024-02-16
9.5 percent is done
7
retrieving 2024-02-23
11.1 percent is done
8
retrieving 2024-03-01
12.7 percent is done
9
retrieving 2024-03-08
14.3 percent is done
10
retrieving 2024-03-15
15.9 percent is done
11
retrieving 2024-03-22
17.5 percent is done
12
retrieving 2024-04-05
19.0 percent is done
13
retrieving 2024-04-12
20.6 percent is done
14
retrieving 2024-04-19
22.2 percent is done
15
retrieving 2024-04-26
23.8 percent is done
16
retrieving 2024-05-03
25.4 percent is done
17
retrieving 2024-05-10
27.0 percent is done
18
retrieving 2024-05-17
28.6 percent is done
19
retrieving 2024-05-24
30.2 percent is done
20
retrieving 2024-05-31
31.7 percent is done
21
retrieving 2024-06-07
33.3 percent is done
22
re

In [289]:
len(df_friday_cut_off) == len(fridays)

True

In [273]:
# Dimension checking purpose
# for i in range(len(fridays)):
#     date_to_cmp = fridays[i]
#     if df_friday_cut_off['date'].dt.date.isin([date_to_cmp]).sum() == 0:
#         print(date_to_cmp)
#         print(i)

2024-11-29
46


In [291]:
df_friday_prior = data_filter(df_all_meta_stock, fridays, specific_time_before = time(14, 30, 0) )

0
retrieving 2024-01-05
0.0 percent is done
300
retrieving 2024-01-12
1.6 percent is done
600
retrieving 2024-01-19
3.2 percent is done
900
retrieving 2024-01-26
4.8 percent is done
1200
retrieving 2024-02-02
6.3 percent is done
1500
retrieving 2024-02-09
7.9 percent is done
1800
retrieving 2024-02-16
9.5 percent is done
2100
retrieving 2024-02-23
11.1 percent is done
2400
retrieving 2024-03-01
12.7 percent is done
2700
retrieving 2024-03-08
14.3 percent is done
3000
retrieving 2024-03-15
15.9 percent is done
3300
retrieving 2024-03-22
17.5 percent is done
3600
retrieving 2024-04-05
19.0 percent is done
3900
retrieving 2024-04-12
20.6 percent is done
4200
retrieving 2024-04-19
22.2 percent is done
4500
retrieving 2024-04-26
23.8 percent is done
4800
retrieving 2024-05-03
25.4 percent is done
5100
retrieving 2024-05-10
27.0 percent is done
5400
retrieving 2024-05-17
28.6 percent is done
5700
retrieving 2024-05-24
30.2 percent is done
6000
retrieving 2024-05-31
31.7 percent is done
6300


In [292]:
len(df_friday_prior)

18900

In [293]:
df_friday_after = data_filter(df_all_meta_stock, fridays, specific_time_after = time(14, 30, 0) )

0
retrieving 2024-01-05
0.0 percent is done
90
retrieving 2024-01-12
1.6 percent is done
180
retrieving 2024-01-19
3.2 percent is done
270
retrieving 2024-01-26
4.8 percent is done
360
retrieving 2024-02-02
6.3 percent is done
450
retrieving 2024-02-09
7.9 percent is done
540
retrieving 2024-02-16
9.5 percent is done
630
retrieving 2024-02-23
11.1 percent is done
720
retrieving 2024-03-01
12.7 percent is done
810
retrieving 2024-03-08
14.3 percent is done
900
retrieving 2024-03-15
15.9 percent is done
990
retrieving 2024-03-22
17.5 percent is done
1080
retrieving 2024-04-05
19.0 percent is done
1170
retrieving 2024-04-12
20.6 percent is done
1260
retrieving 2024-04-19
22.2 percent is done
1350
retrieving 2024-04-26
23.8 percent is done
1440
retrieving 2024-05-03
25.4 percent is done
1530
retrieving 2024-05-10
27.0 percent is done
1620
retrieving 2024-05-17
28.6 percent is done
1710
retrieving 2024-05-24
30.2 percent is done
1800
retrieving 2024-05-31
31.7 percent is done
1890
retrievin

In [294]:
df_friday_after_stat = df_friday_after.groupby('Date_Only').std()[['volume','average']]

In [295]:
df_friday_prior_stat = df_friday_prior.groupby('Date_Only').std()[['volume','average']]

In [296]:
df_friday_prior_stat

,volume,average
Date_Only,,
2024-01-05,17696.394029,0.991232
2024-01-12,26026.915082,0.938764
2024-01-19,52199.124509,1.061027
2024-01-26,16632.938901,1.189034
2024-02-02,173337.414347,4.432709
...,...,...
2025-02-28,25648.103499,3.258261
2025-03-07,22301.877563,8.661127
2025-03-14,20142.479920,2.831638


In [297]:
df_friday_after_stat.columns = ['after_volume', 'target_std']

In [298]:
df_friday_after_stat

,after_volume,target_std
Date_Only,,
2024-01-05,33647.684047,0.647952
2024-01-12,14637.650523,0.623070
2024-01-19,30805.581046,0.491669
2024-01-26,22934.530121,0.797657
2024-02-02,58477.049646,2.300481
...,...,...
2025-02-28,28897.762393,2.193834
2025-03-07,19665.738949,1.290368
2025-03-14,21608.461949,0.717556


In [299]:
df_master = df_friday_after_stat.copy()

In [300]:
df_master['cut_off_price']=df_friday_cut_off['open'].values

In [307]:
df_friday_1_hr_prior = data_filter(df_all_meta_stock, fridays, on_time = time(13, 30, 0) )

0
retrieving 2024-01-05
0.0 percent is done
1
retrieving 2024-01-12
1.6 percent is done
2
retrieving 2024-01-19
3.2 percent is done
3
retrieving 2024-01-26
4.8 percent is done
4
retrieving 2024-02-02
6.3 percent is done
5
retrieving 2024-02-09
7.9 percent is done
6
retrieving 2024-02-16
9.5 percent is done
7
retrieving 2024-02-23
11.1 percent is done
8
retrieving 2024-03-01
12.7 percent is done
9
retrieving 2024-03-08
14.3 percent is done
10
retrieving 2024-03-15
15.9 percent is done
11
retrieving 2024-03-22
17.5 percent is done
12
retrieving 2024-04-05
19.0 percent is done
13
retrieving 2024-04-12
20.6 percent is done
14
retrieving 2024-04-19
22.2 percent is done
15
retrieving 2024-04-26
23.8 percent is done
16
retrieving 2024-05-03
25.4 percent is done
17
retrieving 2024-05-10
27.0 percent is done
18
retrieving 2024-05-17
28.6 percent is done
19
retrieving 2024-05-24
30.2 percent is done
20
retrieving 2024-05-31
31.7 percent is done
21
retrieving 2024-06-07
33.3 percent is done
22
re

In [308]:
df_friday_2_hr_prior = data_filter(df_all_meta_stock, fridays, on_time = time(12, 30, 0) )
df_friday_3_hr_prior = data_filter(df_all_meta_stock, fridays, on_time = time(11, 30, 0) )

0
retrieving 2024-01-05
0.0 percent is done
1
retrieving 2024-01-12
1.6 percent is done
2
retrieving 2024-01-19
3.2 percent is done
3
retrieving 2024-01-26
4.8 percent is done
4
retrieving 2024-02-02
6.3 percent is done
5
retrieving 2024-02-09
7.9 percent is done
6
retrieving 2024-02-16
9.5 percent is done
7
retrieving 2024-02-23
11.1 percent is done
8
retrieving 2024-03-01
12.7 percent is done
9
retrieving 2024-03-08
14.3 percent is done
10
retrieving 2024-03-15
15.9 percent is done
11
retrieving 2024-03-22
17.5 percent is done
12
retrieving 2024-04-05
19.0 percent is done
13
retrieving 2024-04-12
20.6 percent is done
14
retrieving 2024-04-19
22.2 percent is done
15
retrieving 2024-04-26
23.8 percent is done
16
retrieving 2024-05-03
25.4 percent is done
17
retrieving 2024-05-10
27.0 percent is done
18
retrieving 2024-05-17
28.6 percent is done
19
retrieving 2024-05-24
30.2 percent is done
20
retrieving 2024-05-31
31.7 percent is done
21
retrieving 2024-06-07
33.3 percent is done
22
re

In [469]:
df_friday_end = data_filter(df_all_meta_stock, fridays, on_time = time(15, 59, 0) )

0
retrieving 2024-01-05
0.0 percent is done
1
retrieving 2024-01-12
1.6 percent is done
2
retrieving 2024-01-19
3.2 percent is done
3
retrieving 2024-01-26
4.8 percent is done
4
retrieving 2024-02-02
6.3 percent is done
5
retrieving 2024-02-09
7.9 percent is done
6
retrieving 2024-02-16
9.5 percent is done
7
retrieving 2024-02-23
11.1 percent is done
8
retrieving 2024-03-01
12.7 percent is done
9
retrieving 2024-03-08
14.3 percent is done
10
retrieving 2024-03-15
15.9 percent is done
11
retrieving 2024-03-22
17.5 percent is done
12
retrieving 2024-04-05
19.0 percent is done
13
retrieving 2024-04-12
20.6 percent is done
14
retrieving 2024-04-19
22.2 percent is done
15
retrieving 2024-04-26
23.8 percent is done
16
retrieving 2024-05-03
25.4 percent is done
17
retrieving 2024-05-10
27.0 percent is done
18
retrieving 2024-05-17
28.6 percent is done
19
retrieving 2024-05-24
30.2 percent is done
20
retrieving 2024-05-31
31.7 percent is done
21
retrieving 2024-06-07
33.3 percent is done
22
re

In [276]:
pd.set_option('display.max_rows', 10)

In [230]:
for i in range(len(fridays)):
    date_to_cmp = fridays[i]
    if df_all_meta_stock['date'].dt.date.isin([date_to_cmp]).sum() == 0:
        print(date_to_cmp)
        print(i)

2024-03-29
12


In [301]:
df_master['1st_open_price']=df_friday_openning['open'].values

In [309]:
df_master['1hr_prior_price']=df_friday_1_hr_prior['open'].values

In [311]:
df_master['2hr_prior_price']=df_friday_2_hr_prior['open'].values

In [312]:
df_master['3hr_prior_price']=df_friday_3_hr_prior['open'].values

In [470]:
df_master['close_price']=df_friday_end['close'].values

In [314]:
df_friday_prior_stat.columns = ['prior_volume', 'prior_std']

In [315]:
df_friday_prior_stat

,prior_volume,prior_std
Date_Only,,
2024-01-05,17696.394029,0.991232
2024-01-12,26026.915082,0.938764
2024-01-19,52199.124509,1.061027
2024-01-26,16632.938901,1.189034
2024-02-02,173337.414347,4.432709
...,...,...
2025-02-28,25648.103499,3.258261
2025-03-07,22301.877563,8.661127
2025-03-14,20142.479920,2.831638


In [471]:
df_master_w_features = df_master.copy()

In [472]:
df_master_w_features = pd.concat([df_master_w_features,df_friday_prior_stat], axis=1)

In [473]:
df_master_w_features

,after_volume,target_std,cut_off_price,1st_open_price,1hr_prior_price,2hr_prior_price,3hr_prior_price,close_price,prior_volume,prior_std
Date_Only,,,,,,,,,,
2024-01-05,33647.684047,0.647952,351.97,346.99,351.19,350.92,352.00,351.95,17696.394029,0.991232
2024-01-12,14637.650523,0.623070,375.35,370.16,374.64,373.78,376.28,374.49,26026.915082,0.938764
2024-01-19,30805.581046,0.491669,383.87,379.00,383.14,381.14,381.32,383.52,52199.124509,1.061027
2024-01-26,22934.530121,0.797657,395.75,393.85,395.86,396.02,396.27,394.14,16632.938901,1.189034
2024-02-02,58477.049646,2.300481,479.47,459.69,476.81,481.79,477.02,474.88,173337.414347,4.432709
...,...,...,...,...,...,...,...,...,...,...
2025-02-28,28897.762393,2.193834,657.21,658.39,658.33,660.72,658.91,668.53,25648.103499,3.258261
2025-03-07,19665.738949,1.290368,626.71,625.74,615.67,606.18,607.29,625.34,22301.877563,8.661127
2025-03-14,21608.461949,0.717556,605.67,603.00,604.83,605.13,603.48,607.35,20142.479920,2.831638


In [474]:
df_master_w_features['diff_cut_close'] = df_master_w_features['close_price'] - df_master_w_features['cut_off_price']

In [391]:
df_master_w_features['diff_open_cut'] = df_master_w_features['cut_off_price'] - df_master_w_features['1st_open_price']

In [392]:
df_master_w_features['diff_1hr_cut'] = df_master_w_features['cut_off_price'] - df_master_w_features['1hr_prior_price']

In [393]:
df_master_w_features['diff_2hr_cut'] = df_master_w_features['cut_off_price'] - df_master_w_features['2hr_prior_price']

In [394]:
df_master_w_features['diff_3hr_cut'] = df_master_w_features['cut_off_price'] - df_master_w_features['3hr_prior_price']

In [475]:
df_master_w_features

,after_volume,target_std,cut_off_price,1st_open_price,1hr_prior_price,2hr_prior_price,3hr_prior_price,close_price,prior_volume,prior_std,diff_cut_close
Date_Only,,,,,,,,,,,
2024-01-05,33647.684047,0.647952,351.97,346.99,351.19,350.92,352.00,351.95,17696.394029,0.991232,-0.02
2024-01-12,14637.650523,0.623070,375.35,370.16,374.64,373.78,376.28,374.49,26026.915082,0.938764,-0.86
2024-01-19,30805.581046,0.491669,383.87,379.00,383.14,381.14,381.32,383.52,52199.124509,1.061027,-0.35
2024-01-26,22934.530121,0.797657,395.75,393.85,395.86,396.02,396.27,394.14,16632.938901,1.189034,-1.61
2024-02-02,58477.049646,2.300481,479.47,459.69,476.81,481.79,477.02,474.88,173337.414347,4.432709,-4.59
...,...,...,...,...,...,...,...,...,...,...,...
2025-02-28,28897.762393,2.193834,657.21,658.39,658.33,660.72,658.91,668.53,25648.103499,3.258261,11.32
2025-03-07,19665.738949,1.290368,626.71,625.74,615.67,606.18,607.29,625.34,22301.877563,8.661127,-1.37
2025-03-14,21608.461949,0.717556,605.67,603.00,604.83,605.13,603.48,607.35,20142.479920,2.831638,1.68


In [366]:
Meta_data['date']=Meta_data.index

In [367]:
Meta_data

Price,Close,High,Low,Open,Volume,Diff_Price,Vix,Nas_open,Nas_close,Nas_diff,diff_1_Open,diff_2_Open,diff_3_Open,Nas_volume,date
Ticker,META,META,META,META,META,,,,,,,,,,
date,,,,,,,,,,,,,,,
2024-01-02,344.665619,351.503388,338.415079,349.672023,19042200,-5.006404,13.200000,14873.700195,14765.940430,-107.759766,NaN,NaN,NaN,5791170000,2024-01-02
2024-01-03,342.854156,346.317843,341.570199,343.361774,15451100,-0.507617,14.040000,14641.469727,14592.209961,-49.259766,-6.310249,NaN,NaN,5449950000,2024-01-03
2024-01-04,345.491699,346.516866,341.789148,342.883994,12099900,2.607705,14.130000,14532.230469,14510.299805,-21.930664,-0.477780,-6.788029,NaN,5289220000,2024-01-04
2024-01-05,350.299042,351.841759,344.635731,345.362287,13920700,4.936755,13.350000,14500.110352,14524.070312,23.959961,2.478293,2.000513,-4.309736,5232820000,2024-01-05
2024-01-08,356.977539,357.296045,350.398531,353.036124,13890200,3.941415,13.080000,14564.469727,14843.769531,279.299805,7.673837,10.152130,9.674350,6337770000,2024-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-24,618.849976,622.539978,612.200012,614.969971,15741300,3.880005,17.480000,18046.189453,18188.589844,142.400391,31.549988,32.209961,30.689941,6787790000,2025-03-24
2025-03-25,626.309998,633.880005,621.179993,626.760010,15312500,-0.450012,17.150000,18207.970703,18271.859375,63.888672,11.790039,43.340027,44.000000,6475790000,2025-03-25


In [368]:
df_other_features = data_filter(Meta_data, fridays )

0
retrieving 2024-01-05
0.0 percent is done
1
retrieving 2024-01-12
1.6 percent is done
2
retrieving 2024-01-19
3.2 percent is done
3
retrieving 2024-01-26
4.8 percent is done
4
retrieving 2024-02-02
6.3 percent is done
5
retrieving 2024-02-09
7.9 percent is done
6
retrieving 2024-02-16
9.5 percent is done
7
retrieving 2024-02-23
11.1 percent is done
8
retrieving 2024-03-01
12.7 percent is done
9
retrieving 2024-03-08
14.3 percent is done
10
retrieving 2024-03-15
15.9 percent is done
11
retrieving 2024-03-22
17.5 percent is done
12
retrieving 2024-04-05
19.0 percent is done
13
retrieving 2024-04-12
20.6 percent is done
14
retrieving 2024-04-19
22.2 percent is done
15
retrieving 2024-04-26
23.8 percent is done
16
retrieving 2024-05-03
25.4 percent is done
17
retrieving 2024-05-10
27.0 percent is done
18
retrieving 2024-05-17
28.6 percent is done
19
retrieving 2024-05-24
30.2 percent is done
20
retrieving 2024-05-31
31.7 percent is done
21
retrieving 2024-06-07
33.3 percent is done
22
re

In [376]:
df_trimmed_other_featutres = df_other_features.drop(columns=['High','Close','Low','Volume','Nas_close','Open','Diff_Price'])

In [381]:
Nasdaq_data

Price,Close,High,Low,Open,Volume,Nas_1d_diff,Nas_2d_diff,Nas_3d_diff
Ticker,^IXIC,^IXIC,^IXIC,^IXIC,^IXIC,,,
Date,,,,,,,,
2024-01-02,14765.940430,14887.799805,14682.379883,14873.700195,5791170000,NaN,NaN,NaN
2024-01-03,14592.209961,14694.580078,14577.440430,14641.469727,5449950000,-232.230469,NaN,NaN
2024-01-04,14510.299805,14632.769531,14504.780273,14532.230469,5289220000,-109.239258,-341.469727,NaN
2024-01-05,14524.070312,14625.190430,14477.570312,14500.110352,5232820000,-32.120117,-141.359375,-373.589844
2024-01-08,14843.769531,14846.900391,14560.200195,14564.469727,6337770000,64.359375,32.239258,-77.000000
...,...,...,...,...,...,...,...,...
2025-03-24,18188.589844,18210.050781,18030.679688,18046.189453,6787790000,526.349609,460.128906,456.070312
2025-03-25,18271.859375,18281.130859,18170.419922,18207.970703,6475790000,161.781250,688.130859,621.910156


In [377]:
df_trimmed_other_featutres

Price,Vix,Nas_open,Nas_diff,diff_1_Open,diff_2_Open,diff_3_Open,Nas_volume,date,Date_Only
Ticker,,,,,,,,,
0,13.350000,14500.110352,23.959961,2.478293,2.000513,-4.309736,5232820000,2024-01-05,2024-01-05
1,12.700000,15001.009766,-28.250000,-1.960733,9.943156,13.695472,4958940000,2024-01-12,2024-01-12
2,13.300000,15122.379883,188.589844,7.474786,12.640437,5.324905,5604950000,2024-01-19,2024-01-19
3,13.260000,15474.849609,-19.489258,4.160372,4.329594,9.684374,4510530000,2024-01-26,2024-01-26
4,13.850000,15403.160156,225.790039,65.352005,70.268841,55.747290,4800690000,2024-02-02,2024-02-02
...,...,...,...,...,...,...,...,...,...
58,19.629999,18477.169922,370.109375,-24.388334,-1.608621,-7.922955,8247520000,2025-02-28,2025-02-28
59,23.370001,18029.849609,166.371094,-22.609893,-15.616154,-20.122103,8135420000,2025-03-07,2025-03-07
60,21.770000,17523.300781,230.789062,-16.898667,-26.540125,8.278916,7093850000,2025-03-14,2025-03-14


In [373]:
df_master_w_features

,after_volume,target_std,cut_off_price,1st_open_price,1hr_prior_price,2hr_prior_price,3hr_prior_price,prior_volume,prior_std,diff_open_cut,diff_1hr_cut,diff_2hr_cut,diff_3hr_cut
Date_Only,,,,,,,,,,,,,
2024-01-05,33647.684047,0.647952,351.97,346.99,351.19,350.92,352.00,17696.394029,0.991232,4.98,0.78,1.05,-0.03
2024-01-12,14637.650523,0.623070,375.35,370.16,374.64,373.78,376.28,26026.915082,0.938764,5.19,0.71,1.57,-0.93
2024-01-19,30805.581046,0.491669,383.87,379.00,383.14,381.14,381.32,52199.124509,1.061027,4.87,0.73,2.73,2.55
2024-01-26,22934.530121,0.797657,395.75,393.85,395.86,396.02,396.27,16632.938901,1.189034,1.90,-0.11,-0.27,-0.52
2024-02-02,58477.049646,2.300481,479.47,459.69,476.81,481.79,477.02,173337.414347,4.432709,19.78,2.66,-2.32,2.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-28,28897.762393,2.193834,657.21,658.39,658.33,660.72,658.91,25648.103499,3.258261,-1.18,-1.12,-3.51,-1.70
2025-03-07,19665.738949,1.290368,626.71,625.74,615.67,606.18,607.29,22301.877563,8.661127,0.97,11.04,20.53,19.42
2025-03-14,21608.461949,0.717556,605.67,603.00,604.83,605.13,603.48,20142.479920,2.831638,2.67,0.84,0.54,2.19


In [476]:
inter_df = df_master_w_features.copy()
inter_df

,after_volume,target_std,cut_off_price,1st_open_price,1hr_prior_price,2hr_prior_price,3hr_prior_price,close_price,prior_volume,prior_std,diff_cut_close
Date_Only,,,,,,,,,,,
2024-01-05,33647.684047,0.647952,351.97,346.99,351.19,350.92,352.00,351.95,17696.394029,0.991232,-0.02
2024-01-12,14637.650523,0.623070,375.35,370.16,374.64,373.78,376.28,374.49,26026.915082,0.938764,-0.86
2024-01-19,30805.581046,0.491669,383.87,379.00,383.14,381.14,381.32,383.52,52199.124509,1.061027,-0.35
2024-01-26,22934.530121,0.797657,395.75,393.85,395.86,396.02,396.27,394.14,16632.938901,1.189034,-1.61
2024-02-02,58477.049646,2.300481,479.47,459.69,476.81,481.79,477.02,474.88,173337.414347,4.432709,-4.59
...,...,...,...,...,...,...,...,...,...,...,...
2025-02-28,28897.762393,2.193834,657.21,658.39,658.33,660.72,658.91,668.53,25648.103499,3.258261,11.32
2025-03-07,19665.738949,1.290368,626.71,625.74,615.67,606.18,607.29,625.34,22301.877563,8.661127,-1.37
2025-03-14,21608.461949,0.717556,605.67,603.00,604.83,605.13,603.48,607.35,20142.479920,2.831638,1.68


In [ ]:
# Yahoo Multi index issues

In [457]:
df_reset_drop = df_trimmed_other_featutres.reset_index(drop=True)

In [458]:
df_reset_drop.set_index('Date_Only', inplace=True)

In [461]:
df_reset_drop.columns=df_reset_drop.columns.droplevel(1)

In [462]:
df_reset_drop

Price,Vix,Nas_open,Nas_diff,diff_1_Open,diff_2_Open,diff_3_Open,Nas_volume,date
Date_Only,,,,,,,,
2024-01-05,13.350000,14500.110352,23.959961,2.478293,2.000513,-4.309736,5232820000,2024-01-05
2024-01-12,12.700000,15001.009766,-28.250000,-1.960733,9.943156,13.695472,4958940000,2024-01-12
2024-01-19,13.300000,15122.379883,188.589844,7.474786,12.640437,5.324905,5604950000,2024-01-19
2024-01-26,13.260000,15474.849609,-19.489258,4.160372,4.329594,9.684374,4510530000,2024-01-26
2024-02-02,13.850000,15403.160156,225.790039,65.352005,70.268841,55.747290,4800690000,2024-02-02
...,...,...,...,...,...,...,...,...
2025-02-28,19.629999,18477.169922,370.109375,-24.388334,-1.608621,-7.922955,8247520000,2025-02-28
2025-03-07,23.370001,18029.849609,166.371094,-22.609893,-15.616154,-20.122103,8135420000,2025-03-07
2025-03-14,21.770000,17523.300781,230.789062,-16.898667,-26.540125,8.278916,7093850000,2025-03-14


In [436]:
df_reset_drop

Price,Vix,Nas_open,Nas_diff,diff_1_Open,diff_2_Open,diff_3_Open,Nas_volume,date
Ticker,,,,,,,,
Date_Only,,,,,,,,
2024-01-05,13.350000,14500.110352,23.959961,2.478293,2.000513,-4.309736,5232820000,2024-01-05
2024-01-12,12.700000,15001.009766,-28.250000,-1.960733,9.943156,13.695472,4958940000,2024-01-12
2024-01-19,13.300000,15122.379883,188.589844,7.474786,12.640437,5.324905,5604950000,2024-01-19
2024-01-26,13.260000,15474.849609,-19.489258,4.160372,4.329594,9.684374,4510530000,2024-01-26
2024-02-02,13.850000,15403.160156,225.790039,65.352005,70.268841,55.747290,4800690000,2024-02-02
...,...,...,...,...,...,...,...,...
2025-02-28,19.629999,18477.169922,370.109375,-24.388334,-1.608621,-7.922955,8247520000,2025-02-28
2025-03-07,23.370001,18029.849609,166.371094,-22.609893,-15.616154,-20.122103,8135420000,2025-03-07


In [477]:
df_final_target_features = pd.concat([inter_df,df_reset_drop], axis=1)

In [478]:
df_final_target_features

,after_volume,target_std,cut_off_price,1st_open_price,1hr_prior_price,2hr_prior_price,3hr_prior_price,close_price,prior_volume,prior_std,diff_cut_close,Vix,Nas_open,Nas_diff,diff_1_Open,diff_2_Open,diff_3_Open,Nas_volume,date
Date_Only,,,,,,,,,,,,,,,,,,,
2024-01-05,33647.684047,0.647952,351.97,346.99,351.19,350.92,352.00,351.95,17696.394029,0.991232,-0.02,13.350000,14500.110352,23.959961,2.478293,2.000513,-4.309736,5232820000,2024-01-05
2024-01-12,14637.650523,0.623070,375.35,370.16,374.64,373.78,376.28,374.49,26026.915082,0.938764,-0.86,12.700000,15001.009766,-28.250000,-1.960733,9.943156,13.695472,4958940000,2024-01-12
2024-01-19,30805.581046,0.491669,383.87,379.00,383.14,381.14,381.32,383.52,52199.124509,1.061027,-0.35,13.300000,15122.379883,188.589844,7.474786,12.640437,5.324905,5604950000,2024-01-19
2024-01-26,22934.530121,0.797657,395.75,393.85,395.86,396.02,396.27,394.14,16632.938901,1.189034,-1.61,13.260000,15474.849609,-19.489258,4.160372,4.329594,9.684374,4510530000,2024-01-26
2024-02-02,58477.049646,2.300481,479.47,459.69,476.81,481.79,477.02,474.88,173337.414347,4.432709,-4.59,13.850000,15403.160156,225.790039,65.352005,70.268841,55.747290,4800690000,2024-02-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-28,28897.762393,2.193834,657.21,658.39,658.33,660.72,658.91,668.53,25648.103499,3.258261,11.32,19.629999,18477.169922,370.109375,-24.388334,-1.608621,-7.922955,8247520000,2025-02-28
2025-03-07,19665.738949,1.290368,626.71,625.74,615.67,606.18,607.29,625.34,22301.877563,8.661127,-1.37,23.370001,18029.849609,166.371094,-22.609893,-15.616154,-20.122103,8135420000,2025-03-07
2025-03-14,21608.461949,0.717556,605.67,603.00,604.83,605.13,603.48,607.35,20142.479920,2.831638,1.68,21.770000,17523.300781,230.789062,-16.898667,-26.540125,8.278916,7093850000,2025-03-14


In [479]:
df_final_target_features.columns

Index(['after_volume', 'target_std', 'cut_off_price', '1st_open_price',
       '1hr_prior_price', '2hr_prior_price', '3hr_prior_price', 'close_price',
       'prior_volume', 'prior_std', 'diff_cut_close', 'Vix', 'Nas_open',
       'Nas_diff', 'diff_1_Open', 'diff_2_Open', 'diff_3_Open', 'Nas_volume',
       'date'],
      dtype='object')

In [480]:
df_final_target_features.to_csv('final_target_features.csv', index=False)
df_final_target_features.to_pickle('final_target_features.pkl')